In [ ]:
!pip install nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')

#Veliko hvala freeCodeCamp, Youtube i naravno StackOverflow


import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from string import punctuation

# Učitavanje podataka
df = pd.read_csv('disaster-tweets.csv').dropna()

# Priprema podataka
stop_words = set(stopwords.words('english')) | set(punctuation)
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    filtered = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
    return filtered

df['processed'] = df['text'].apply(preprocess)

# Kreiranje vokabulara
all_words = [word for tokens in df['processed'] for word in tokens]
word_freq = nltk.FreqDist(all_words)
common_words = word_freq.most_common(10000)
vocab = [word[0] for word in common_words]

# Kreiranje BoW feature vektora
def create_features(tokens):
    features = np.zeros(len(vocab), dtype=np.float32)
    for token in tokens:
        if token in vocab:
            features[vocab.index(token)] += 1
    return features

X = np.array([create_features(tokens) for tokens in df['processed']])
y = df['target'].values

# Podela podataka na trening i test skupove "ručno"
def train_test_split_manual(X, y, test_size=0.2):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split_idx = int(X.shape[0] * (1-test_size))
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2)

# Implementacija Naive Bayes klasifikatora
class NaiveBayesClassifier:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Inicijalizacija parametara
        self.priors = np.zeros(n_classes, dtype=np.float64)
        self.likelihoods = np.zeros((n_classes, n_features), dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.priors[idx] = X_c.shape[0] / float(n_samples)
            self.likelihoods[idx, :] = (np.sum(X_c, axis=0) + 1) / (np.sum(X_c) + n_features)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            likelihood = np.sum(np.log(self.likelihoods[idx]) * x)
            posterior = prior + likelihood
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

# Treniranje i evaluacija modela
model = NaiveBayesClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Ručno izračunavanje tačnosti
def accuracy_score_manual(y_true, y_pred):
    correct = np.sum(y_true == y_pred)
    return correct / len(y_true)

accuracy = accuracy_score_manual(y_test, predictions)
print(f'Accuracy: {accuracy}')

# Analiza reči i LR metrike ostaje ista
positive_words = [word for tokens in df[df['target'] == 1]['processed'] for word in tokens]
negative_words = [word for tokens in df[df['target'] == 0]['processed'] for word in tokens]
positive_freq = nltk.FreqDist(positive_words)
negative_freq = nltk.FreqDist(negative_words)
print("Najčešće reči u pozitivnim tvitovima:", positive_freq.most_common(5))
print("Najčešće reči u negativnim tvitovima:", negative_freq.most_common(5))

# Izračunavanje LR metrike
lr_metric = {word: positive_freq[word] / negative_freq[word] for word in set(positive_freq) & set(negative_freq) if positive_freq[word] >= 10 and negative_freq[word] >= 10}
sorted_lr = sorted(lr_metric.items(), key=lambda item: -item[1])
print("Top 5 reči sa najvećom LR vrednošću:", sorted_lr[:5])
print("Top 5 reči sa najmanjom LR vrednošću:", sorted_lr[-5:])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.7874015748031497
Najčešće reči u pozitivnim tvitovima: [('http', 1661), ('fire', 177), ('bomb', 111), ('kill', 101), ('via', 87)]
Najčešće reči u negativnim tvitovima: [('http', 1492), ('like', 195), ('get', 145), ('new', 132), ('amp', 132)]
Top 5 reči sa najvećom LR vrednošću: [('kill', 8.416666666666666), ('train', 5.230769230769231), ('report', 4.8), ('evacu', 3.9444444444444446), ('famili', 3.619047619047619)]
Top 5 reči sa najmanjom LR vrednošću: [('feel', 0.22916666666666666), ('want', 0.2), ('love', 0.18309859154929578), ('obliter', 0.1694915254237288), ('scream', 0.15151515151515152)]


Na osnovu datih ispisanih rijesenja dolazimo do iducih zakljucaka:
Model jako lose radi pri prisustvu linkova u tvitu, jer veliki broj tvitova u katastrofama zapravo koristi linkove na neke poznate novinarske sajtove nas porter stemmer prilikom ucitavanja linka ce ga samo skratiti na http, zato u oba slucaja pozitivnih i negativnih tvitoa rijec http je najcesca
Naprotiv logicnom razmisljanju dolazimo do iducih saznjanja vezanih za zapravo ostale rijeci koje se cesto pojavljuju:
Rijeci koje u svom obicnom bez-kontekstnom znacaju cesto imaju vezu za negativnim mislima ili losim stvarima poput fire, bomb, kill se zapravo cesto pojavjluju u pozitivnim tvitovima sto znaci da prosta analiza na osnovu samo rijec kao rijeci je nedovoljna vec da je kontekst jako bitan za bilo kakvo stvaranje predstave o znacaju tvita. Naizgled potpuno slican fenomen je primjetan u slucaju negativnih tvitova gdje se pojavljuju like, get, new, i stemovano "amp" sve rijeci za koje bi smo rekli da su pozitivne ali naizgled prkose ocekivanju.
Bitna napomena jeste da je nas klasifikator nakon 5 ponovljenih pokretanja dosao do neke srednje vrijednosti od ~75% tacnosti tako da cak iako je 3/4 jako dobro , greske su moguce.
Zatim vezano za LR vrijednosti tu zapravo vidimo kako visi LR pozitivno utice na izbor rijeci za klasifikovanje u pozitivne ili negativne tvitove a manja LR vrijednsot suprotno. Znajuci ovo u daljem radu sa analizom teksta bi bilo pametno uzimati LR u obzir kao neki vid konteksta odnosno koliko neke rijeci mogu kontekse recenice ili smisao da promjene na dobro ili na lose (vidno za ceste rijeci)

